# Redes Neuronales

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
import os
import pickle

* Cargamos .csv y separamos "X" e "y" . Recordemos "X" son todas las columnas excepto la columna target en cambio "y" es la columna target.

In [2]:
df = pd.read_csv('datos_limpios_modelo_10cols.csv')


X = df.drop(columns=['precio_contado'])
y = df['precio_contado']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


* Creamos un modelo de red neuronal con varias capas:
  - La primera capa recibe los datos de entrada (el numero de caracteristicas que tiene cada ejemplo).
  - Luego hay dos capas intermedias (ocultas) con varias neuronas que ayudan a que el modelo aprenda patrones complejos.
  - La ultima capa tiene una neurona que nos da la prediccion del precio.

* Preparamos el modelo para entrenarlo:
  - Usamos **Adam** para ajustar el modelo.
  - Le decimos que mida que tan bien esta haciendo la prediccion con  **MSE** (error cuadratico medio).
  - Tambien le pedimos que nos muestre **MAE** (error absoluto medio) para entender mejor su rendimiento.


In [3]:
model_mse = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model_mse.compile(optimizer=Adam(), loss='mse', metrics=['mae'])



2025-06-24 19:55:16.817914: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


* Entrenamos el modelo usando los datos de entrenamiento ("X_train" y "y_train").
* Tambien usamos los datos de prueba ("X_test" y "y_test") para validar como esta funcionando el modelo mientras aprende.
* Definimos que el entrenamiento dure 100 epocas (pasadas completas sobre todos los datos).
* En cada epoca, el modelo procesa los datos en grupos de 32 ejemplos (batch size).
* "verbose=1" hace que se muestre una barra de progreso durante el entrenamiento.


In [4]:
history_mse = model_mse.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)



Epoch 1/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 7327932416.0000 - mae: 29494.4785 - val_loss: 3445460992.0000 - val_mae: 15611.6270
Epoch 2/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 2787899136.0000 - mae: 15950.2295 - val_loss: 2984452608.0000 - val_mae: 15055.0420
Epoch 3/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2622671360.0000 - mae: 14837.2588 - val_loss: 2765142528.0000 - val_mae: 13774.5986
Epoch 4/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2438168576.0000 - mae: 13587.9121 - val_loss: 2595771904.0000 - val_mae: 12730.1191
Epoch 5/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 2775557120.0000 - mae: 12656.1338 - val_loss: 2444128768.0000 - val_mae: 11696.0156
Epoch 6/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 1649797760.0000 - mae: 11503.5742 - val_loss: 2278457600.0000 - val_mae: 11191.8008
Epoch 7/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 1598390016.0000 - mae: 10955.6973 - val_loss: 213832729

* Creamos un grafico para mostrar como evoluciona la perdida del modelo a lo largo de las epocas.
* Añadimos dos lineas:
  - La perdida en los datos de entrenamiento ("loss").
  - La perdida en los datos de validación ("val_loss").
* Esto nos ayuda a ver si el modelo esta aprendiendo bien o si esta sobreajustando.
* El eje X muestra las epocas y el eje Y la magnitud de la perdida (MSE).


In [5]:
fig_mse = go.Figure()
fig_mse.add_trace(go.Scatter(y=history_mse.history['loss'], name='Entrenamiento (MSE)'))
fig_mse.add_trace(go.Scatter(y=history_mse.history['val_loss'], name='Validación (MSE)'))
fig_mse.update_layout(title='Pérdida con función MSE', xaxis_title='Épocas', yaxis_title='MSE')
fig_mse.show()

* En el gráfico, tanto la perdida en entrenamiento como en validacion bajan rapidamente y se acercan bastante desde las primeras epocas. Esto indica que el modelo esta aprendiendo de manera eficiente y generaliza bien, es decir, no solo memoriza los datos de entrenamiento, sino que tambien funciona bien con datos nuevos. La convergencia rapida y cercana de ambas curvas sugiere que el modelo esta bien ajustado, sin problemas evidentes de sobreajuste o subajuste.

Repetimos el mismo proceso de creacion, entrenamiento y evaluacion del modelo, pero esta vez usando la funcion de perdida MAE para observar como se comporta la curva de error absoluto medio durante el entrenamiento y la validacion

In [6]:
model_mae = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model_mae.compile(optimizer=Adam(), loss='mae', metrics=['mse'])


In [7]:
history_mae = model_mae.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)


Epoch 1/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 27804.4297 - mse: 6003122688.0000 - val_loss: 12022.2256 - val_mse: 3824688896.0000
Epoch 2/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 10771.5479 - mse: 2766640128.0000 - val_loss: 7725.9917 - val_mse: 2415149056.0000
Epoch 3/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 7553.9575 - mse: 1604435840.0000 - val_loss: 6073.0312 - val_mse: 1747995264.0000
Epoch 4/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 5532.9517 - mse: 624726784.0000 - val_loss: 4428.3511 - val_mse: 1199275136.0000
Epoch 5/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 4029.2607 - mse: 385783168.0000 - val_loss: 3712.2129 - val_mse: 978550464.0000
Epoch 6/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 3479.5559 - mse: 354738944.0000 - val_loss: 3533.1938 - val_mse: 919412160.0000
Epoch 7/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 3278.8223 - mse: 188293536.0000 - val_loss: 3446.9399 - val_mse: 901805696.0

In [8]:
fig_mae = go.Figure()
fig_mae.add_trace(go.Scatter(y=history_mae.history['loss'], name='Entrenamiento (MAE)'))
fig_mae.add_trace(go.Scatter(y=history_mae.history['val_loss'], name='Validación (MAE)'))
fig_mae.update_layout(title='Pérdida con función MAE', xaxis_title='Épocas', yaxis_title='MAE')
fig_mae.show()


* En este grafico, ambas lineas —entrenamiento y validacion— se acercan y disminuyen juntas de manera constante, lo que indica que el modelo esta aprendiendo correctamente y no presenta sobreajuste (overfitting). Esto significa que el modelo generaliza bien y su rendimiento es estable tanto con los datos de entrenamiento como con datos nuevos.


In [14]:
model_mae.save('modelos/modelo_red_neuronal_mae.keras')
